In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import nltk , re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.metrics import accuracy_score

In [3]:
!git clone https://github.com/mamintoosi-cs/aws-machine-learning-university-accelerated-nlp.git

fatal: destination path 'aws-machine-learning-university-accelerated-nlp' already exists and is not an empty directory.


In [4]:
%cd aws-machine-learning-university-accelerated-nlp

/content/aws-machine-learning-university-accelerated-nlp


In [5]:
train_df = pd.read_csv('data/final_project/imdb_train.csv', header=0)
train_df.head()
print('the shape of Train set : ', train_df.shape)

the shape of Train set :  (25000, 2)


In [6]:
print(train_df.isna().sum())

text     0
label    0
dtype: int64


In [7]:
test_df = pd.read_csv('data/final_project/imdb_test.csv', header=0)
test_df.head()
print('the shape of test set : ', test_df.shape)

the shape of test set :  (25000, 2)


In [8]:
# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't",
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

# Stemmer
snow = SnowballStemmer('english')

def process_text(texts):
    final_text_list=[]
    for sent in texts:

        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ""

        filtered_sentence=[]

        sent = sent.lower() # Lowercase
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:

        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words

        final_text_list.append(final_string)

    return final_text_list

In [9]:
print("Processing the reviewText fields")
train_text_list = process_text(train_df["text"].tolist())
test_text_list = process_text(test_df["text"].tolist())

Processing the reviewText fields


In [10]:
tfidf_vectorizer = TfidfVectorizer(max_features=80)
tfidf_vectorizer.fit(train_text_list)

tfidf_X_train =tfidf_vectorizer.transform(train_text_list).toarray()
y_train = train_df['label']

print('tfidf_X_train shape : ', tfidf_X_train.shape)
print('y_train shape : ', y_train.shape)

tfidf_X_train shape :  (25000, 80)
y_train shape :  (25000,)


In [11]:
knn = KNeighborsClassifier()
knn.fit(tfidf_X_train, y_train)

KNeighborsClassifier()

In [12]:
tfidf_X_test =tfidf_vectorizer.transform(test_text_list).toarray()
y_test = test_df['label']

print('tfidf_X_test shape : ',tfidf_X_test.shape)
print('y_test shape : ', y_test.shape)

y_pred = knn.predict(tfidf_X_test)

tfidf_accuracy = accuracy_score(y_test, y_pred)
print (tfidf_accuracy)

tfidf_X_test shape :  (25000, 80)
y_test shape :  (25000,)
0.65816


In [13]:
tfidf_vectorizer = TfidfVectorizer(max_features=80)
tfidf_vectorizer.fit(test_text_list)

tfidf_X_test =tfidf_vectorizer.transform(test_text_list).toarray()
y_test = test_df['label']

print('tfidf_X_test shape : ',tfidf_X_test.shape)
print('y_test shape : ', y_test.shape)

y_pred = knn.predict(tfidf_X_test)

tfidf_accuracy = accuracy_score(y_test, y_pred)
print (tfidf_accuracy)

tfidf_X_test shape :  (25000, 80)
y_test shape :  (25000,)
0.50992


In [14]:
binary_vectorizer = CountVectorizer(binary=True)
binary_vectorizer.fit(train_text_list)

binary_X_train =binary_vectorizer.transform(train_text_list).toarray()
y_train = train_df['label']

print('tfidf_X_train shape : ', binary_X_train.shape)
print('y_train shape : ', y_train.shape)

tfidf_X_train shape :  (25000, 57654)
y_train shape :  (25000,)


In [15]:
b_knn = KNeighborsClassifier()
b_knn.fit(binary_X_train, y_train)

KNeighborsClassifier()

In [16]:
binary_X_test =binary_vectorizer.transform(test_text_list).toarray()
y_test = test_df['label']

print('tfidf_X_test shape : ',binary_X_test.shape)
print('y_test shape : ', y_test.shape)

y_pred = knn.predict(tfidf_X_test)

binary_accuracy = accuracy_score(y_test, y_pred)
print (binary_accuracy)

tfidf_X_test shape :  (25000, 57654)
y_test shape :  (25000,)
0.50992


In [ ]:
binary_vectorizer = CountVectorizer(binary=True)
binary_vectorizer.fit(test_text_list)

binary_X_test =binary_vectorizer.transform(test_text_list).toarray()
y_test = test_df['label']

print('tfidf_X_test shape : ',binary_X_test.shape)
print('y_test shape : ', y_test.shape)

y_pred = knn.predict(tfidf_X_test)

binary_accuracy = accuracy_score(y_test, y_pred)
print (binary_accuracy)

In [ ]:
if binary_accuracy > tfidf_accuracy :
  print ('binary_accuracy is larger than tfidf_accuracy ')
else :
  print ( 'tfidf_accuracy is larger than binary_accuracy' )